In [ ]:
#Load the libraries
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import nltk
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import LabelBinarizer
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from wordcloud import WordCloud,STOPWORDS
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize,sent_tokenize
from bs4 import BeautifulSoup
import spacy
import re,string,unicodedata
from nltk.tokenize.toktok import ToktokTokenizer
from nltk.stem import LancasterStemmer,WordNetLemmatizer
from sklearn.linear_model import LogisticRegression,SGDClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC
from textblob import TextBlob
from textblob import Word
from sklearn.metrics import classification_report,confusion_matrix,accuracy_score
from gensim.models import Word2Vec
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, classification_report



In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from google.colab import files



# After uploading, you can load the file with pandas
file_name = "/content/test.csv"  # Make sure this matches the name of the uploaded file
df = pd.read_csv(file_name)

print(df.head())  # Verify the data is loaded correctly


                                              review sentiment
0  Steven Rea plays a forensic scientist thrust o...  positive
1  As the first of the TV specials offered on the...  positive
2  There may something poetically right in seeing...  negative
3  all i can say about this film is to read the b...  negative
4  I thought it was a pretty good movie and shoul...  positive


In [ ]:
from google.colab import files



# After uploading, you can load the file with pandas
file_name = "/content/train.csv"  # Make sure this matches the name of the uploaded file
df_train = pd.read_csv(file_name)

print(df_train.head())  # Verify the data is loaded correctly


                                              review sentiment
0  SAPS AT SEA <br /><br />Aspect ratio: 1.37:1<b...  negative
1  If you want mindless action, hot chicks and a ...  positive
2  "The Woman in Black" is easily one of the cree...  positive
3  I can barely find the words to describe how mu...  negative
4  What's in here ?! Let me tell you. It's the pr...  negative


In [ ]:
df.describe()

,review,sentiment
count,20000,20000
unique,19926,2
top,"After some internet surfing, I found the ""Home...",positive
freq,2,10065


In [ ]:
df_train.describe()

,review,sentiment
count,30000,30000
unique,29853,2
top,Loved today's show!!! It was a variety and not...,negative
freq,3,15065


In [ ]:
df_train['sentiment'].value_counts()

negative    15065
positive    14935
Name: sentiment, dtype: int64

In [ ]:
df['sentiment'].value_counts()

positive    10065
negative     9935
Name: sentiment, dtype: int64

In [ ]:
import nltk
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


True

In [ ]:
def preprocess_text(text, method='stemming'):
    # Define the tokenizer
    tokenizer = ToktokTokenizer()

    # Define stopword list
    stopword_list = nltk.corpus.stopwords.words('english')

    # Removing HTML tags
    soup = BeautifulSoup(text, "html.parser")
    text = soup.get_text()

    # Removing text inside square brackets
    text = re.sub('\[[^]]*\]', '', text)

    # Define function for removing special characters
    def remove_special_characters(text, remove_digits=True):
        pattern = r'[^a-zA-z0-9\s]'
        text = re.sub(pattern, '', text)
        return text

    # Remove special characters
    text = remove_special_characters(text)

    # Tokenization
    tokens = tokenizer.tokenize(text)
    tokens = [token.strip() for token in tokens]

    # Stemming or Lemmatization
    if method == 'stemming':
        stemmer = PorterStemmer()
        tokens = [stemmer.stem(token) for token in tokens]
    elif method == 'lemmatization':
        lemmatizer = WordNetLemmatizer()
        tokens = [lemmatizer.lemmatize(token) for token in tokens]

    # Removing the stopwords
    filtered_tokens = [token for token in tokens if token.lower() not in stopword_list]

    # Reconstruct the text
    text = ' '.join(filtered_tokens)

    return text

# Example usage
method = 'lemmatization' # or 'stemming'
df_train['review'] = df_train['review'].apply(lambda x: preprocess_text(x, method=method))
df['review'] = df['review'].apply(lambda x: preprocess_text(x, method=method))


<ipython-input-5-bf4cf3b991ac>:9: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  soup = BeautifulSoup(text, "html.parser")


In [ ]:
#Count vectorizer for bag of words
cv=CountVectorizer(max_features = 50000)
#transformed train reviews
cv_train_reviews=cv.fit_transform(df_train['review'])
cv_test_reviews=cv.transform(df['review'])


print('BOW_cv_train:',cv_train_reviews.shape)




print('BOW_cv_test:',cv_test_reviews.shape)

#vocab=cv.get_feature_names()-toget feature names

BOW_cv_train: (30000, 50000)
BOW_cv_test: (20000, 50000)


In [ ]:
tv=TfidfVectorizer(max_features = 50000)
#transformed train reviews
tv_train_reviews=tv.fit_transform(df_train['review'])
tv_test_reviews=tv.transform(df['review'])


print('Tfidf_train:',tv_train_reviews.shape)
print('Tfidf_test:',tv_test_reviews.shape)

Tfidf_train: (30000, 50000)
Tfidf_test: (20000, 50000)


In [ ]:
#labeling the sentient data
lb=LabelBinarizer()
#transformed sentiment data
sentiment_data=lb.fit_transform(df_train['sentiment'])
sentiment_data_test=lb.transform(df['sentiment'])
print(sentiment_data.shape)
print(sentiment_data_test.shape)

(30000, 1)
(20000, 1)


In [ ]:
#Spliting the sentiment data
sentiment_data=sentiment_data[:30000]
sentiment_data_test=sentiment_data_test[:20000]

print(sentiment_data)
print(sentiment_data_test)

[[0]
 [1]
 [1]
 ...
 [1]
 [0]
 [0]]
[[1]
 [1]
 [0]
 ...
 [0]
 [1]
 [0]]


In [ ]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
!pip install gensim nltk


DOC2VEC

In [ ]:
import nltk
from gensim.models import Doc2Vec
from gensim.models.doc2vec import TaggedDocument
import numpy as np

# Tokenize text data and tag each document for Doc2Vec
tagged_train_reviews = [TaggedDocument(words=nltk.word_tokenize(text), tags=[i]) for i, text in enumerate(df_train['review'])]
tagged_test_reviews = [TaggedDocument(words=nltk.word_tokenize(text), tags=[i]) for i, text in enumerate(df['review'])]

# Train a Doc2Vec model
doc2vec_model = Doc2Vec(vector_size=200, window=5, min_count=2, workers=4, epochs=20)
doc2vec_model.build_vocab(tagged_train_reviews)
doc2vec_model.train(tagged_train_reviews, total_examples=doc2vec_model.corpus_count, epochs=doc2vec_model.epochs)

# Function to infer the document vector
def infer_vector(doc2vec_model, doc):
    return doc2vec_model.infer_vector(doc)

# Vectorize text data using the embeddings
train_vectors_doc = np.array([infer_vector(doc2vec_model, review.words) for review in tagged_train_reviews])
test_vectors_doc = np.array([infer_vector(doc2vec_model, review.words) for review in tagged_test_reviews])

# Check the shapes of the vectors
print('Doc2Vec_train:', train_vectors_doc.shape)
print('Doc2Vec_test:', test_vectors_doc.shape)


Doc2Vec_train: (30000, 200)
Doc2Vec_test: (20000, 200)


WORD2VEC

In [ ]:
# Tokenize text data for Word2Vec
tokenized_train_reviews = [nltk.word_tokenize(text) for text in df_train['review']]
tokenized_test_reviews = [nltk.word_tokenize(text) for text in df['review']]

# Train a Word2Vec model
w2v_model = Word2Vec(sentences=tokenized_train_reviews, vector_size=300, window=8, min_count=5, workers=4, sg=1, epochs=20)

# Define a function to get the document vector
def document_vector(word2vec_model, doc):
    # Remove out-of-vocabulary words
    doc = [word for word in doc if word in word2vec_model.wv.key_to_index]
    # If the document contains no words in the model's vocabulary, return a zero vector
    if not doc:
        return np.zeros(word2vec_model.vector_size)
    # Compute the mean of the word vectors for words in the document
    return np.mean([word2vec_model.wv[word] for word in doc], axis=0)

# Vectorize text data using the embeddings

train_vectors = np.array([document_vector(w2v_model, review) for review in tokenized_train_reviews])
test_vectors = np.array([document_vector(w2v_model, review) for review in tokenized_test_reviews])

# Check the shapes of the vectors
print('Word2Vec_train:', train_vectors.shape)
print('Word2Vec_test:', test_vectors.shape)

Word2Vec_train: (30000, 300)
Word2Vec_test: (20000, 300)


WORD2VEC WITH TFIDF

In [ ]:
import numpy as np
import nltk
from gensim.models import Word2Vec
from sklearn.feature_extraction.text import TfidfVectorizer

# Assuming df_train['review'] and df['review'] are your datasets

# Tokenize text data for Word2Vec
tokenized_train_reviews = [nltk.word_tokenize(text) for text in df_train['review']]
tokenized_test_reviews = [nltk.word_tokenize(text) for text in df['review']]

# Train a Word2Vec model
w2v_model = Word2Vec(sentences=tokenized_train_reviews, vector_size=300, window=8, min_count=5, workers=4, sg=1, epochs=20)

# Compute TF-IDF
tfidf = TfidfVectorizer(analyzer=lambda x: x)
tfidf.fit([' '.join(review) for review in tokenized_train_reviews])
max_idf = max(tfidf.idf_)
word_weights = {word: tfidf.idf_[i] if i in tfidf.vocabulary_ else max_idf for i, word in enumerate(tfidf.get_feature_names_out())}

# Modify the document_vector function to use TF-IDF weights
def document_vector_tfidf(word2vec_model, doc, word_weights):
    # Remove out-of-vocabulary words
    doc = [word for word in doc if word in word2vec_model.wv.key_to_index]
    if not doc:
        return np.zeros(word2vec_model.vector_size)
    # Compute the weighted mean of the word vectors for words in the document
    return np.mean([word2vec_model.wv[word] * word_weights.get(word, 1.0) for word in doc], axis=0)

# Vectorize text data using the embeddings with TF-IDF
train_vectors = np.array([document_vector_tfidf(w2v_model, review, word_weights) for review in tokenized_train_reviews])
test_vectors = np.array([document_vector_tfidf(w2v_model, review, word_weights) for review in tokenized_test_reviews])

# Check the shapes of the vectors
print('Word2Vec_train:', train_vectors.shape)
print('Word2Vec_test:', test_vectors.shape)


Word2Vec_train: (30000, 300)
Word2Vec_test: (20000, 300)


In [ ]:
def load_glove_embeddings(path):
    embeddings_dict = {}
    with open(path, 'r', encoding='utf-8') as f:
        for line in f:
            values = line.split()
            word = values[0]
            vector = np.asarray(values[1:], "float32")
            embeddings_dict[word] = vector
    return embeddings_dict

# Specify the path to the GloVe file
glove_path = '/content/glove.6B.100d.txt'  # Update this path
glove_embeddings = load_glove_embeddings(glove_path)


In [ ]:
def document_vector_glove(glove_embeddings, doc_tokens):
    """Generate a document vector using GloVe embeddings."""
    vectors = [glove_embeddings[word] for word in doc_tokens if word in glove_embeddings]
    if vectors:
        return np.mean(vectors, axis=0)
    else:
        return np.zeros(100)  # Return a zero vector if none of the words are in the GloVe vocabulary

# Tokenize text data for GloVe
tokenized_train_reviews = [nltk.word_tokenize(text) for text in df_train['review']]
tokenized_test_reviews = [nltk.word_tokenize(text) for text in df['review']]

# Assuming 'tokenized_train_reviews' and 'tokenized_test_reviews' are available from your previous code
train_vectors_glove = np.array([document_vector_glove(glove_embeddings, review) for review in tokenized_train_reviews])
test_vectors_glove = np.array([document_vector_glove(glove_embeddings, review) for review in tokenized_test_reviews])



In [ ]:
import time

# Start time for training
start_time = time.time()

# Instantiate and train the Random Forest classifier
rf_model = RandomForestClassifier()
rf_model.fit(train_vectors_glove, sentiment_data.ravel())  # Ensure labels are in correct shape

# End time for training
end_time = time.time()

# Total time for training
training_time = end_time - start_time

# Predict on test data
start_time = time.time()  # Start time for prediction
rf_predictions = rf_model.predict(test_vectors_glove)  # Ensure this is your test dataset
end_time = time.time()  # End time for prediction

# Total time for prediction
prediction_time = end_time - start_time

# Evaluate performance
print("Random Forest Accuracy:", accuracy_score(sentiment_data_test.ravel(), rf_predictions))
print("Random Forest Classification Report:\n", classification_report(sentiment_data_test.ravel(), rf_predictions))
print("Time taken for training: {:.2f} seconds".format(training_time))
print("Time taken for prediction: {:.2f} seconds".format(prediction_time))


Random Forest Accuracy: 0.76995
Random Forest Classification Report:
               precision    recall  f1-score   support

           0       0.76      0.78      0.77      9935
           1       0.78      0.76      0.77     10065

    accuracy                           0.77     20000
   macro avg       0.77      0.77      0.77     20000
weighted avg       0.77      0.77      0.77     20000

Time taken for training: 40.83 seconds
Time taken for prediction: 0.38 seconds


In [ ]:
import xgboost as xgb

# Start time for training
start_time = time.time()

# Instantiate XGBoost classifier
xgb_model = xgb.XGBClassifier()

# Train the XGBoost classifier
xgb_model.fit(train_vectors_glove, sentiment_data.ravel())

# End time for training
end_time = time.time()

# Total time for training
training_time = end_time - start_time

# Predict on test data
start_time = time.time()  # Start time for prediction
xgb_predictions = xgb_model.predict(test_vectors_glove)
end_time = time.time()  # End time for prediction

# Total time for prediction
prediction_time = end_time - start_time

# Evaluate performance
print("XGBoost Accuracy:", accuracy_score(sentiment_data_test.ravel(), xgb_predictions))
print("XGBoost Classification Report:\n", classification_report(sentiment_data_test.ravel(), xgb_predictions))
print("Time taken for training: {:.2f} seconds".format(training_time))
print("Time taken for prediction: {:.2f} seconds".format(prediction_time))


XGBoost Accuracy: 0.78765
XGBoost Classification Report:
               precision    recall  f1-score   support

           0       0.78      0.79      0.79      9935
           1       0.79      0.78      0.79     10065

    accuracy                           0.79     20000
   macro avg       0.79      0.79      0.79     20000
weighted avg       0.79      0.79      0.79     20000

Time taken for training: 12.33 seconds
Time taken for prediction: 0.13 seconds


In [ ]:
start_time = time.time()
# Instantiate and train the k-NN classifier
knn_model = KNeighborsClassifier(n_neighbors=150)
knn_model.fit(train_vectors_glove, sentiment_data.ravel())  # Ensure labels are in correct shape
# End time for training
end_time = time.time()

# Total time for training
training_time = end_time - start_time

start_time = time.time()  # Start time for prediction
# Predict on test data
knn_predictions = knn_model.predict(test_vectors_glove)
end_time = time.time()  # End time for prediction

# Total time for prediction
prediction_time = end_time - start_time


# Evaluate performance
print("k-NN Accuracy:", accuracy_score(sentiment_data_test.ravel(), knn_predictions))
print("k-NN Classification Report:\n", classification_report(sentiment_data_test.ravel(), knn_predictions))
print("Time taken for training: {:.2f} seconds".format(training_time))
print("Time taken for prediction: {:.2f} seconds".format(prediction_time))

k-NN Accuracy: 0.74935
k-NN Classification Report:
               precision    recall  f1-score   support

           0       0.71      0.83      0.77      9935
           1       0.80      0.67      0.73     10065

    accuracy                           0.75     20000
   macro avg       0.76      0.75      0.75     20000
weighted avg       0.76      0.75      0.75     20000

Time taken for training: 0.01 seconds
Time taken for prediction: 10.78 seconds


In [ ]:
from sklearn.naive_bayes import GaussianNB

In [ ]:
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score
import time

# Initialize the Gaussian Naive Bayes model
gnb = GaussianNB()

# Start timing for training
start_time_train = time.time()

# Fitting the GNB model with training data
gnb_fit = gnb.fit(train_vectors_glove, sentiment_data.ravel())  # Ensure sentiment_data is a 1D array

# End timing for training
end_time_train = time.time()

# Calculate the elapsed time for training
training_time = end_time_train - start_time_train

# Start timing for prediction
start_time_predict = time.time()

# Model prediction on the test data
gnb_predict = gnb.predict(test_vectors_glove)

# End timing for prediction
end_time_predict = time.time()

# Calculate the elapsed time for prediction
prediction_time = end_time_predict - start_time_predict

# Accuracy score for the Gaussian Naive Bayes model
gnb_score = accuracy_score(sentiment_data_test, gnb_predict)

# Print results
print(gnb_fit)
print(gnb_predict)
print(f"Time taken for model to train: {training_time} seconds")
print(f"Time taken for model to predict: {prediction_time} seconds")
print("GNB Score:", gnb_score)


GaussianNB()
[1 1 0 ... 0 0 1]
Time taken for model to train: 0.06059980392456055 seconds
Time taken for model to predict: 0.022963285446166992 seconds
GNB Score: 0.72095


In [ ]:
#training the model
mnb=MultinomialNB()
#fitting the svm for bag of words
mnb_bow=mnb.fit(cv_train_reviews,sentiment_data)
print(mnb_bow)


/usr/local/lib/python3.10/dist-packages/sklearn/utils/validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


MultinomialNB()


In [ ]:
#training the model
mnb=MultinomialNB()
#fitting the svm for bag of words
mnb_bow=mnb.fit(tv_train_reviews,sentiment_data)
print(mnb_bow)

/usr/local/lib/python3.10/dist-packages/sklearn/utils/validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


MultinomialNB()


In [ ]:
mnb_bow_predict=mnb.predict(cv_test_reviews)
print(mnb_bow_predict)

[0 1 0 ... 0 1 1]


In [ ]:
mnb_bow_predict=mnb.predict(tv_test_reviews)
print(mnb_bow_predict)

[0 1 0 ... 0 0 0]


In [ ]:
#Accuracy score for bag of words
mnb_bow_score=accuracy_score(test_sentiments,mnb_bow_predict)
print("mnb_bow_score :",mnb_bow_score)

TypeError: Labels in y_true and y_pred should be of the same type. Got y_true=['negative' 'positive'] and y_pred=[0 1]. Make sure that the predictions provided by the classifier coincides with the true labels.

In [ ]:
#Classification report for bag of words
mnb_bow_report=classification_report(test_sentiments,mnb_bow_predict,target_names=['Positive','Negative'])
print(mnb_bow_report)

              precision    recall  f1-score   support

    Positive       0.65      0.59      0.62      9935
    Negative       0.63      0.69      0.66     10065

    accuracy                           0.64     20000
   macro avg       0.64      0.64      0.64     20000
weighted avg       0.64      0.64      0.64     20000



In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from torch.nn.functional import softmax
import numpy as np
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import torch

# Load tokenizer and model
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased-finetuned-sst-2-english")
model = AutoModelForSequenceClassification.from_pretrained("distilbert-base-uncased-finetuned-sst-2-english")

# Ensure the model is on the correct device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

def predict_with_distilbert(reviews, batch_size=20000):
    model.eval()  # Set model to evaluation mode
    predictions = []

    for i in range(0, len(reviews), batch_size):
        batch_reviews = reviews[i:i+batch_size]
        inputs = tokenizer(batch_reviews, padding=True, truncation=True, return_tensors="pt").to(device)

        with torch.no_grad():
            outputs = model(**inputs)
            probs = softmax(outputs.logits, dim=-1)
            batch_predictions = torch.argmax(probs, dim=1)
            predictions.extend(batch_predictions.cpu().numpy())

    return np.array(predictions)

# Assuming `test_reviews` is a list of preprocessed text data from your test dataset
test_reviews_list = test_reviews.tolist()
distilbert_predictions = predict_with_distilbert(test_reviews_list, batch_size=8)  # Adjust batch_size as needed

# Convert predictions to match your original sentiment labels if necessary

# Calculate evaluation metrics for DistilBERT
accuracy_distilbert = accuracy_score(sentiment_data_test, distilbert_predictions)
precision_distilbert = precision_score(sentiment_data_test, distilbert_predictions)
recall_distilbert = recall_score(sentiment_data_test, distilbert_predictions)
f1_distilbert = f1_score(sentiment_data_test, distilbert_predictions)

# Print metrics for comparison
print(f"DistilBERT Accuracy: {accuracy_distilbert}")
print(f"DistilBERT Precision: {precision_distilbert}")
print(f"DistilBERT Recall: {recall_distilbert}")
print(f"DistilBERT F1 Score: {f1_distilbert}")



DistilBERT Accuracy: 0.8891
DistilBERT Precision: 0.9142645971914265
DistilBERT Recall: 0.860307998012916
DistilBERT F1 Score: 0.8864660114660114
